# Análise do Ebit nas empresas da B3

<b>Ebit</b> - Lucro antes dos juros e tributos (LAJIR)

Demonstra o <b><i>lucro operacional</i></b> da empresa, excluindo as despesas ou receitas financeiras para chegar ao valor aproximado do lucro operacional da empresa em seus resultados operacionais

Considerado indicador de desempenho e mede a performance operacional da empresa

<b> Fórmula </b>

(+) Resultado Líquido (Lucro/Prejuízo Consolidado do Período)

(+) Encargos Financeiros de Financimento (Resultado Financeiro)

(+) Impostos sobre os lucros (Imposto de Renda e Contribuição Social sobre o Lucro)

(=) Ebit

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pandas_datareader as web
import yfinance as yfinance
import warnings
warnings.filterwarnings('ignore')

Definir caminho para salvar arquivos

In [2]:
caminho_arquivos = '../dados/cvm'

In [3]:
# Carregar DRE
dre = pd.read_csv(f'{caminho_arquivos}/DRE.csv', sep=';', encoding='ISO-8859-1', decimal=',')

In [4]:
cnpj = '47.960.950/0001-21'
empresa = 'MGLU3'
dre_empresa = dre[dre['CNPJ_CIA'] == cnpj]

In [ ]:
df_ebit = dre_empresa[(dre_empresa['CD_CONTA'] == '3.05')]

df_ebit['TRIMESTRE'] = 0
df_ebit.loc[(pd.to_datetime(df_ebit['DT_INI_EXERC']).dt.month == 1) & 
    (pd.to_datetime(df_ebit['DT_FIM_EXERC']).dt.month == 3), 'TRIMESTRE'] = 1
df_ebit.loc[(pd.to_datetime(df_ebit['DT_INI_EXERC']).dt.month == 1) & 
    (pd.to_datetime(df_ebit['DT_FIM_EXERC']).dt.month == 6), 'TRIMESTRE'] = 2
df_ebit.loc[(pd.to_datetime(df_ebit['DT_INI_EXERC']).dt.month == 1) & 
    (pd.to_datetime(df_ebit['DT_FIM_EXERC']).dt.month == 9), 'TRIMESTRE'] = 3
df_ebit.loc[(pd.to_datetime(df_ebit['DT_INI_EXERC']).dt.month == 1) & 
    (pd.to_datetime(df_ebit['DT_FIM_EXERC']).dt.month == 12), 'TRIMESTRE'] = 4

df_ebit = df_ebit[df_ebit['TRIMESTRE'] != 0]
df_ebit['DT_FIM_EXERC'] = pd.to_datetime(df_ebit['DT_FIM_EXERC']) 

In [ ]:
df_ebit = df_ebit[['DT_FIM_EXERC','VL_CONTA']]
df_ebit.columns = ['Date','Ebit']
df_ebit.set_index('Date', inplace=True)

In [ ]:
df_tmp = df_ebit[df_ebit.index.year > 2011]
df_tmp['ANO'] = df_tmp.index.year
f, ax = plt.subplots(figsize=(15, 7))
sns.barplot(x="ANO",y="Ebit", palette="ch:.25", data=df_tmp)

In [ ]:
df_cotacao = pd.DataFrame(web.get_data_yahoo(f"{empresa}.SA", start="2010-01-01", end="2021-12-30")["Close"])
df_cotacao.columns = ['Preco']

In [ ]:
df_empresa = pd.concat([df_ebit,df_cotacao], axis=1)
df_empresa['Preco'].fillna(method='ffill', inplace=True)
df_empresa.dropna(inplace=True)

In [ ]:
f, ax = plt.subplots(figsize=(15, 7))
sns.lineplot(palette="ch:.25", data=df_empresa['Ebit'])

In [ ]:
f, ax = plt.subplots(figsize=(15, 7))
sns.lineplot(palette="ch:.25", data=df_empresa['Preco'])

In [ ]:
df_empresa['Var_Ebit'] = ((df_empresa['Ebit']/df_empresa['Ebit'].shift(1)) - 1)*100
df_empresa['Var_Preco'] = ((df_empresa['Preco']/df_empresa['Preco'].shift(1)) - 1)*100

In [ ]:
df_empresa

In [ ]:
f, ax = plt.subplots(figsize=(15, 7))
sns.lineplot(palette="ch:.25", data=df_empresa[['Var_Ebit','Var_Preco']])

In [ ]:
df_empresa.corr()

In [ ]:
df_empresa['Preco_anterior'] = df_empresa['Preco'].shift(1)
df_empresa['Ebit_anterior'] = df_empresa['Ebit'].shift(1)

In [ ]:
df_empresa